# Head

In [1]:
# head cell
import json
import pandas as pd
import numpy as np

with open('breakdown/misc.json') as file:
    raw = json.load(file)

# Monster Names & IDs
SECTIONS:
- **monster_names**: rise english names
- **monster_names_mr**: sunbreak english names
- **monsters**: IDs
- **random_mystery_enemy**: monster A★ and M★ rank numbers

COLUMNS:
- **id**: base monster id
- **sub_id**: indicates monster variant
- **em_id** (enemy_type): monster id when referenced by "EnemyIndex###(_MR)"
- **em_type.Em**: internal monster identifier
- **mon_name**: monster name
- **normal_rank**: M★ rank. 12 indicates access through anomaly only.
- **mystery_rank**: A★ rank. 12 indicates access as secondary target only.

OUTPUT: **df_monsters**
| id | sub_id | em_id | em_type.Em | mon_name | normal_rank | mystery_rank |
|-|-|-|-|-|-|-|
| 1 | 0 | 0 | 1 | Rathian | 2 | 3 |
| 1 | 2 | 76 | 513 | Gold Rathian | 6 | 7 |
| 1 | 7 | 1 | 1793 | Apex Rathian | 12 | 12 |
| 2 | 0 | 2 | 2 | Rathalos | 4 | 5 |
| 2 | 2 | 77 | 514 | Silver Rathalos | 6 | 7 |

In [2]:
manual_entry = True #maintaints Toadversary and non-anomaly MR monsters in data

with open('breakdown/monsters.json') as file:
    raw = json.load(file)

# Monster Names
sect = raw.get('monster_names').get('entries')
df = pd.json_normalize(sect)
df['mon_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
df['em_id'] = pd.to_numeric(df['name'].apply(lambda x: x[-3:])) #grab only id number
df = df.iloc[0:46] #remove small monsters
df_monster_names = df[['em_id', 'mon_name']].copy()

# MR Monster Names
sect = raw.get('monster_names_mr').get('entries')
df = pd.json_normalize(sect)
df['mon_name'] = df['content'].apply(lambda x: x[1])
df['em_id'] = pd.to_numeric(df['name'].apply(lambda x: x[-6:-3]))
df = pd.concat([df.iloc[0:23], df.iloc[31:]], axis=0, ignore_index=True)

df_monster_names = pd.concat([df_monster_names, df[['em_id', 'mon_name']]], axis=0, ignore_index=True)

# Monster IDs
sect = raw.get('monsters')
df = pd.json_normalize(sect)
df = df[['id','sub_id','enemy_type','em_type.Em']]
df.rename(columns={'enemy_type':'em_id'}, inplace=True)

df_monsters = pd.merge(df,df_monster_names, on='em_id', how='left')

if manual_entry:
    df_monsters.loc[df_monsters['em_id'] == 46, 'mon_name'] = 'Toadversary' # add Toadversary to list
else:
    df_monsters.dropna(subset=['mon_name'], inplace=True) # drop Toadversary from list
    df_monsters.reset_index(drop=True, inplace=True)

# Anomaly Data
sect = raw.get('random_mystery_enemy').get('lot_enemy_list')
df = pd.json_normalize(sect)
df = df[['em_type.Em','normal_rank','mystery_rank']]

df_monsters = pd.merge(df_monsters,df, on='em_type.Em', how='left')

if manual_entry:
    df_monsters.loc[df_monsters['mon_name'] == 'Amatsu', 'normal_rank'] = 6
    df_monsters.loc[df_monsters['mon_name'] == 'Wind Serpent Ibushi', 'normal_rank'] = 6
    df_monsters.loc[df_monsters['mon_name'] == 'Thunder Serpent Narwa', 'normal_rank'] = -1
    df_monsters.loc[df_monsters['mon_name'] == 'Narwa the Allmother', 'normal_rank'] = 6
    df_monsters.loc[df_monsters['mon_name'] == 'Gaismagorm', 'normal_rank'] = 6
else:
    df_monsters.dropna(subset=['mystery_rank'], inplace=True)
    df_monsters.reset_index(drop=True, inplace=True)

df_monsters.fillna(-1, inplace=True)
df_monsters[['normal_rank','mystery_rank']] = df_monsters[['normal_rank','mystery_rank']].astype(int)

df_monsters.to_excel('monster.xlsx')
# print(df_monsters.head())

# Item Names, IDs, & Properties
SECTIONS:
- **items_name_msg**: rise english names & ids
- **items_name_msg_mr**: sunbreak english names & ids
- **items**: item properties

OUTPUT: **df_items**
| item_id | item_name | type_ | category_prefix | material_category | category_worth | rare |
|-|-|-|-|-|-|-|
| 6 | Potion | Consume | 'None' | [] | 0 | 1 |
| 7 | Mega Potion | Consume | 'None' | [] | 0 | 2 |
| 5 | Max Potion | Consume | 'None' | [] | 0 | 3 |
| 467 | Ancient Potion | Consume | 'None' | [] | 0 | 4 |
| 13 | Antidote | Consume | 'None' | [] | 0 | 1 |

In [ ]:
with open('breakdown/items.json') as file:
    raw = json.load(file)

# Item Names & IDs
sect = raw.get('items_name_msg').get('entries')
df = pd.json_normalize(sect)
df.rename(columns={'name':'item_id'}, inplace=True)
df=df[~df['item_id'].str.contains('I_EC_')] # remove endemic life
df['item_id'] = df['item_id'].str.extract('(\d+)', expand=False) # grab only item number
df.dropna(inplace=True) # drop blanks
df['item_id'] = pd.to_numeric(df['item_id'], errors='coerce', downcast='integer') # convert column to int
df['item_name'] = df['content'].apply(lambda x: x[1]) # grab only english name
df = df[~df['item_name'].str.contains('COLOR FF0000', na=False)] # remove placeholder items
df_item_names = df[['item_id', 'item_name']].copy()

# MR Item Names & IDs
sect = raw.get('items_name_msg_mr').get('entries')
df = pd.json_normalize(sect)
df.rename(columns={'name':'item_id'}, inplace=True)
df=df[~df['item_id'].str.contains('I_EC_')]
df['item_id'] = df['item_id'].str.extract('(\d+)', expand=False)
df.dropna(inplace=True)
df['item_id'] = pd.to_numeric(df['item_id'], errors='coerce', downcast='integer')
df['item_name'] = df['content'].apply(lambda x: x[1])
df = df[~df['item_name'].str.contains('COLOR FF0000', na=False)]

df_item_names = pd.concat((df_item_names, df[['item_id', 'item_name']]), axis=0, ignore_index=True)

# Item Types, Categories, Worth, and Rarity
sect = raw.get('items').get('param')
df = pd.json_normalize(sect)
df.rename(columns={'id.Normal':'item_id'}, inplace=True)
df = df[['item_id', 'type_', 'material_category', 'category_worth', 'rare']]
df = pd.merge(df, df_item_names, on='item_id', how='left')
df.dropna(subset=['item_name'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Denesting Category Dictionary
df['category_prefix'] = df['material_category'].apply(lambda x: next(iter(x[0])) if isinstance(x[0],dict) else x[0]) # prefix
df['material_category'] = df['material_category'].apply(lambda x: [d for d in x if isinstance(d, dict)]) # None removal
df['material_category'] = df['material_category'].apply(lambda x: [list(d.values())[0] for d in x] if x else []) # denest

df_items = df[['item_id', 'item_name', 'type_', 'category_prefix', 'material_category', 'category_worth', 'rare']].copy()

print(df_items.head())
# df_items.to_excel('items.xlsx')

## Drop Tables
SECTIONS:
- **reward_id_lot_table**: base game item drop chance and count tables
- **reward_id_lot_table_mr**: sunbreak item drop chance and count tables

OUTPUT: **df_drop_tables**
| drop_table_id | lot_rule | item_id_list | num_list | probability_list |
|-|-|-|-|-|
| 310000 | Random | [125, 126, 687, 128, 699, 131, 719] | [1, 1, 1, 2, 1, 1, 1] | [17, 24, 15, 22, 8, 8, 6] |
| 310001 | Random | [691, 132, 720, 705, 704, 129, 700] | [1, 1, 1, 1, 1, 1, 1] | [9, 14, 7, 19, 25, 14, 12] |
| 310002 | Random | [693, 133, 710, 706, 707, 688, 701] | [1, 1, 1, 1, 1, 1, 1] | [5, 11, 16, 18, 14, 21, 15] |
| 310003 | Random | [702, 484, 686, 689, 127, 133, 721] | [2, 1, 1, 1, 1, 1, 1] | [18, 17, 23, 14, 9, 13, 6] |
| 310004 | Random | [695, 134, 479, 711, 690, 718, 127] | [1, 1, 1, 1, 1, 1, 1] | [5, 9, 19, 19, 14, 23, 11] |

In [ ]:
with open('breakdown/items.json') as file:
    raw = json.load(file)

# Item Drop Tables
sect = raw.get('reward_id_lot_table').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['num_list'] = df['num_list'].apply(lambda y: [x for x in y if x>0])
df['probability_list'] = df['probability_list'].apply(lambda y: [x for x in y if x>0])
df_drop_tables = df.copy()

# MR Item Drop Tables
sect = raw.get('reward_id_lot_table_mr').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['num_list'] = df['num_list'].apply(lambda y: [x for x in y if x>0])
df['probability_list'] = df['probability_list'].apply(lambda y: [x for x in y if x>0])
df_drop_tables = pd.concat([df_drop_tables, df], axis=0, ignore_index=True)
df_drop_tables.rename(columns={'id':'drop_table_id'}, inplace=True)

# df_drop_tables.to_excel('item_drop_tables.xlsx')
print(df_drop_tables.head())

## Anomaly Items, Subset of Items

COLUMNS:
- **item_id**: item ID
- **item_name**: item name
- **type**: material type [aff, ess, coin]
- **category_worth**: item value in augmenting and charm melding

OUTPUT: **df_anomaly_items**
| item_id | item_name | type | category_worth |
|-|-|-|-|
| 2521 | Amber Essence | ess | 10 |
| 2522 | Amber Essence+ | ess | 20 |
| 2523 | Prime Amber Essence | ess | 40 |
| 2584 | Afflicted Bone | aff | 2 |
| 2585| Afflicted Pelt | aff | 2 |

In [ ]:
df = df_items[~df_items['category_prefix'].str.contains('LrHr')] #remove LrHr items so that only Mr or None results show
df_anomaly_items = df[df['material_category'].apply(lambda x: 73 in x)].copy()
df_anomaly_items['type'] = 'aff'
df_ess = df[df['material_category'].apply(lambda x: 74 in x)].copy()
df_ess['type'] = 'ess'
df_coin = df_items[df_items['item_name'].str.contains('Investigation Coin',case=False,na=False)].copy()
df_coin['type'] = 'coin'
df_anomaly_items = pd.concat([df_anomaly_items, df_ess, df_coin], axis=0, ignore_index=True)
df_anomaly_items = df_anomaly_items[['item_id','item_name','type','category_worth']]
df_anomaly_items.reset_index(drop=True, inplace=True)

# df_anomaly_items.to_excel('anom_items.xlsx')
print(df_anomaly_items.head())

### Anomaly Drop Lots
SECTIONS:
- **mystery_reward_item**: anomaly drop lots for quests, investigations, and special investigations

COLUMNS:
- **lv_lower** (~~lv_lower_limit~~): Anomaly investigation reward bracket lower level. 0 for quests. 300 for special investigations.
- **lv_upper** (~~lv_upper_limit~~): Anomaly investigation reward bracket upper level. 0 for quests. 300 for special investigations.
- **em_type.Em**: primary target monster internal ID
- **item_id** (~~reward_item.Normal~~): the primary afflicted material dropped by primary target monster on carve and part break
- **anom_item_chance** (~~hagibui_probability~~): chance of getting primary afflicted material on carve and part break
- **anom_drop_table_id** (~~quest_reward_table_index~~): table ID for quest rewards.
- **anom_drop_lot_ids** (~~additional_quest_reward_table_index~~): additional quest reward table IDs.
- **sp_ess_table_id** (~~special_quest_reward_table_index~~): bonus essence drop table for single AR brackets (e.g. AR200, AR220)
- **multi_ess_table_id** (~~multiple_target_reward_table_index~~): bonus essence drop table for hunting monster as secondary target
- **multi_coin_table_id**: (~~multiple_fix_reward_table_index~~): bonus coin drop table for hunting monster as secondary target
- **mystery_reward_table**: ???

OUTPUT: **df_anom_drop_lots**
| lv_lower | lv_upper | anom_item_chance | anom_drop_table_id | anom_drop_lot_ids |
|-|-|-|-|-|
| 0 | 0 | 40 | 0 | [0, 0, 0, 0] |
| 1 | 30 | 40 | 560000 | [550000, 540000, 0, 330300] |
| 31 | 50 | 40 | 560000 | [550100, 540001, 550000, 330300] |
| 51 | 70 | 40 | 560001 | [550100, 540100, 550000, 330300] |
| 71 | 90 | 40 | 560001 | [550101, 540100, 550000, 330301] |

| sp_ess_table_id | multi_ess_table_id | multi_coin_table_id | em_type.Em | item_id |
|-|-|-|-|-|
| 0 | 0 | 0 | 98 | 2585 |
| 0 | 760000 | 770000 | 98 | 2585 |
| 0 | 760000 | 770000 | 98 | 2885 |
| 0 | 760001 | 770001 | 98 | 2885 |
| 0 | 760001 | 770001 | 98 | 2885 |

In [ ]:
# Anomaly Drop Lots
sect = raw.get('mystery_reward_item').get('param')
df = pd.json_normalize(sect)
df = df[df['is_special_mystery']==False] # remove SI entries since drops are the same as AR300 and represented separately in [300,300] investigations
df = df[~((df['lv_lower_limit']==0) & (df['lv_upper_limit']==0))] # remove 0-0 entries as they don't represent anomaly quest rewards

df['reward_item.Normal'] = pd.to_numeric(df['reward_item.Normal'].fillna(-1), errors='ignore', downcast='integer')
df.drop(['is_special_mystery','quest_no', 'item_num', 'reward_item', 'mystery_reward_table'], axis=1, inplace=True)

df.rename(columns={'lv_lower_limit':'lv_lower',
           'lv_upper_limit':'lv_upper',
           'reward_item.Normal':'item_id',
           'hagibui_probability':'anom_item_chance',
           'quest_reward_table_index':'anom_drop_table_id',
           'additional_quest_reward_table_index':'anom_drop_lot_ids',
           'special_quest_reward_table_index':'sp_ess_table_id',
           'multiple_target_reward_table_index':'multi_ess_table_id',
           'multiple_fix_reward_table_index':'multi_coin_table_id'
           },inplace=True)
df.reset_index(drop=True, inplace=True)

df_anom_drop_lots = df.copy()
print(df_anom_drop_lots.head())

### Anomaly Material Average Item Drop

REFERENCES:
- **df_anomaly_items**
- **df_drop_tables**

COLUMNS:
- **drop_table_id**: drop table ID
- **aff_avg**: average afflicted material melding worth
- **ess_avg**: average essence augmenting worth
- **coin_avg**: average investigation coin worth

OUTPUT: **df_anom_drop_avg**
| drop_table_id | aff_avg | ess_avg | coin_avg |
|-|-|-|-|
| 530000 | 4.8 | 0.0 | 0.0 |
| 530001 | 4.8 | 0.0 | 0.0 |
| 530002 | 9.6 | 0.0 | 0.0 |
| 530003 | 9.6 | 0.0 | 0.0 |
| 530004 | 14.4 | 0.0 | 0.0 |

In [ ]:
#filter out non-anomaly drop tables
mapper = set(df_anomaly_items['item_id'])
df = df_drop_tables[df_drop_tables['item_id_list'].apply(lambda x: any(y in mapper for y in x))]

#add translated names to items
# mapper = dict(zip(df_items['item_id'], df_items['item_name']))
# df_drop_tables['tab']=df_drop_tables['item_id_list'].apply(lambda x: [mapper.get(i,'?') for i in x])

# translate item_id_list into worth_list
df2 = df_anomaly_items[df_anomaly_items['type']=='aff']
mapper = dict(zip(df2['item_id'], df2['category_worth']))
df['aff_worth_list'] = df['item_id_list'].apply(lambda x: [mapper.get(i,0) for i in x])
df2 = df_anomaly_items[df_anomaly_items['type']=='ess']
mapper = dict(zip(df2['item_id'], df2['category_worth']))
df['ess_worth_list'] = df['item_id_list'].apply(lambda x: [mapper.get(i,0) for i in x])
df2 = df_anomaly_items[df_anomaly_items['type']=='coin']
mapper = dict(zip(df2['item_id'], [1]))
df['coin_worth_list'] = df['item_id_list'].apply(lambda x: [mapper.get(i,0) for i in x])

# sum of products: item_cts, item_chance, aff_worth
df['aff_avg'] = df.apply(lambda x: round(np.sum(np.array(x['num_list']) * np.array(x['probability_list']) * np.array(x['aff_worth_list']) /100),2), axis=1)
df['ess_avg'] = df.apply(lambda x: round(np.sum(np.array(x['num_list']) * np.array(x['probability_list']) * np.array(x['ess_worth_list']) /100),2), axis=1)
df['coin_avg'] = df.apply(lambda x: round(np.sum(np.array(x['num_list']) * np.array(x['probability_list']) * np.array(x['coin_worth_list']) /100),2), axis=1)

df_anom_drop_avg = df[['drop_table_id','aff_avg','ess_avg','coin_avg']].copy()

# df_anom_drop_avg['am_avg'] = df_anom_drop_avg.apply(lambda x: [x['aff_avg'], x['ess_avg'], x['coin_avg']], axis=1)
# df_anom_drop_avg = df[['drop_table_id','am_avg']]

# df_anom_drop_avg.to_excel('anom_drop_avg.xlsx')
print(df_anom_drop_avg.head())

### Average Material Drop Per Anomaly Investigation Level Bracket
REFERENCES:
- **df_anom_drop_lots**
- **df_anomaly_items**
- **df_anom_drop_avg**
- **df_monsters**

COLUMNS:
- **lv_lower**: anomaly investigation level bracket lower end
- **lv_upper**: anomaly investigation level bracket upper end
- **item_name**: primary afflicted material reward
- **aff_avg**: average afflicted material augment and melding value
- **ess_avg**: average essence augment value
- **coin_avg**: average investigation coin value
- **ess_avg_multi**: average essence augment value for multi-monster investigations
- **coin_avg_multi**: average investigation coin value for multi-monster investigations
- **mon_name**: primary target name
- **mystery_rank**: primary target rank

OUTPUT: **df_anom_avg**
| lv_lower | lv_upper | item_name | aff_avg | ess_avg | coin_avg | ess_avg_multi | coin_avg_multi | mon_name | mystery_rank |
|-|-|-|-|-|-|-|-|-|-|
| 1 | 30 | Afflicted Pelt | 7.2 | 56.5 | 0.05 | 29 | 2 | Great Izuchi | 1 |
| 31 | 50 | Afflicted Hide+ | 29.6 | 67.5 | 0.05 | 29 | 2 | Great Izuchi | 1 |
| 51 | 70 | Afflicted Hide+ | 29.6 | 115 | 0.25 | 58 | 2.4 | Great Izuchi | 1 |
| 71 | 90 | Afflicted Hide+ | 38.4 | 115 | 0.25 | 58 | 2.4 | Great Izuchi | 1 |
| 91 | 100 | Afflicted Hide+ | 63.8 | 289 | 0.25 | 84 | 2.4 | Great Izuchi | 1 |


In [ ]:
carves = 4 # number of carves + estimated partbreaks
df = df_anom_drop_lots

#replace item_id with item name and worth from df_anomaly_items
df = pd.merge(df, df_anomaly_items[['item_id','item_name','category_worth']], on='item_id', how='left')
df.drop(columns=['item_id'], inplace=True)
df.fillna(0,inplace=True)

# #create carve reward worth column
df['carve_worth'] = df.apply(lambda x: round(x['category_worth']*x['anom_item_chance']*carves/100,2), axis=1)
df.drop(columns=['category_worth','anom_item_chance'], inplace=True)

# #replace reward tables with am, ess, coin worth averages
df = pd.merge(left=df, left_on='anom_drop_table_id', right=df_anom_drop_avg, right_on='drop_table_id', how='left')
df.drop(columns=['anom_drop_table_id','drop_table_id'], inplace=True)
df.fillna(0,inplace=True)
# df = df.rename(columns={'am_avg':'quest_rewards'})
df['aff_avg'] = df['aff_avg']+df['carve_worth']
df.drop(columns='carve_worth', inplace=True)

#replace additional quest rewards with am worth averages
mapper = dict(zip(df_anom_drop_avg['drop_table_id'], list(zip(df_anom_drop_avg['aff_avg'],df_anom_drop_avg['ess_avg'],df_anom_drop_avg['coin_avg']))))
df['aqr'] = df['anom_drop_lot_ids'].apply(lambda x: [mapper.get(i,[0,0,0]) for i in x])
#zip addition quest rewards
df['aqr_sum'] = df['aqr'].apply(lambda x: [round(sum(i),2) for i in zip(*x)])
df.drop(columns=['aqr','anom_drop_lot_ids'], inplace=True)
#individual column sum
df['aff_avg'] = df.apply(lambda x: x['aff_avg']+x['aqr_sum'][0], axis=1)
df['ess_avg'] = df.apply(lambda x: x['ess_avg']+x['aqr_sum'][1], axis=1)
df['coin_avg'] = df.apply(lambda x: x['coin_avg']+x['aqr_sum'][2], axis=1)
df.drop(columns='aqr_sum', inplace=True)

#sp essence rewards
df = pd.merge(left=df, left_on='sp_ess_table_id', right=df_anom_drop_avg[['drop_table_id','ess_avg']], right_on='drop_table_id', how='left', suffixes=['','_sp'])
df.fillna(0,inplace=True)
df['ess_avg'] = df['ess_avg']+df['ess_avg_sp']
df.drop(columns=['drop_table_id','ess_avg_sp','sp_ess_table_id'], inplace=True)

#multi rewards
df = pd.merge(left=df, left_on='multi_ess_table_id', right=df_anom_drop_avg[['drop_table_id','ess_avg']], right_on='drop_table_id', how='left', suffixes=['','_multi'])
df.fillna(0,inplace=True)
df.drop(columns=['multi_ess_table_id','drop_table_id'], inplace=True)
df = pd.merge(left=df, left_on='multi_coin_table_id', right=df_anom_drop_avg[['drop_table_id','coin_avg']], right_on='drop_table_id', how='left', suffixes=['','_multi'])
df.fillna(0,inplace=True)
df.drop(columns=['multi_coin_table_id','drop_table_id'], inplace=True)

#monster names
df = pd.merge(df, df_monsters[['em_type.Em','mon_name','mystery_rank']], on='em_type.Em',how='left')
df.drop(columns=['em_type.Em'], inplace=True)

df_anom_avg = df.copy()
# df_anom_avg.to_excel('anom_avg.xlsx')
print(df_anom_avg.head())

# Quest Names & IDs

SECTIONS:
- **quest_arena_msg**
- **quest_dlc_msg**
- **quest_hall_msg**
- **quest_hall_msg_mr**
- **quest_hall_msg_mr2**
- **quest_tutorial_msg**
- **quest_village_msg**
- **npc_mission_msg**
- **npc_mission_msg_mr**

COLUMNS:
- **quest_id**: internal quest ID
- **quest_type**: quest category
- **quest_name**: quest name

OUTPUT: **df_quest_names**
| quest_id | quest_name |
|-|-|
| 20101 | Arena 01 |
| 20201 | Arena 02 |
| 20301 | Arena 03 |
| 20601 | Arena 04 |
| 20701 | Arena 05 |

OUTPUT2: **df_npc_quest_names**
| quest_id | quest_name |
|-|-|
| 1 | Elder Fugen's Errand |
| 2 | Thick, Gooey Dango |
| 3 | Bunny Dango—with Eggs! |
| 4 | A Dango a Day |
| 5 | Only the Good Eggs |

In [17]:
with open('breakdown/quests.json') as file:
    raw = json.load(file)

# Quest Lists
sections = ['quest_arena_msg',
           'quest_dlc_msg',
           'quest_hall_msg',
           'quest_hall_msg_mr',
           'quest_hall_msg_mr2',
           'quest_tutorial_msg',
           'quest_village_msg',
           'npc_mission_msg',
           'npc_mission_msg_mr']
df_quest_names = pd.DataFrame()
for q in sections:
    sect = raw.get(q).get('entries')
    df = pd.json_normalize(sect)
    df = df[df['name'].str.contains('_01')]
    df['quest_type'] = df['name'].str.extract('(^[A-Za-z]+)')
    df['quest_id'] = df['name'].str.extract('(\d+)')
    df['quest_id'] = pd.to_numeric(df['quest_id'],errors='coerce',downcast='integer')
    df['content'] = df['content'].apply(lambda x: x[1])
    df = df[~df['content'].str.contains('COLOR FF0000', na=False)]
    df_quest_names = pd.concat([df_quest_names,df[['quest_id','quest_type','content']]], axis=0,ignore_index=True)

df_quest_names.rename(columns={'content':'quest_name'}, inplace=True)

df_npc_quest_names = df_quest_names[df_quest_names['quest_type']=='NSQ'].copy()
df_npc_quest_names = df_npc_quest_names[['quest_id','quest_name']]
df_npc_quest_names.reset_index(drop=True, inplace=True)

df_quest_names = df_quest_names[df_quest_names['quest_type']=='QN']
df_quest_names = df_quest_names[['quest_id','quest_name']]
df_quest_names.reset_index(drop=True, inplace=True)

print(df_quest_names)
print(df_npc_quest_names)

     quest_id                         quest_name
0       20101                           Arena 01
1       20201                           Arena 02
2       20301                           Arena 03
3       20601                           Arena 04
4       20701                           Arena 05
..        ...                                ...
698       605                   Thundering Voice
699       606                     Twisted Desire
700       607                  A Test of Courage
701       608          Like a Flash of Lightning
702       609  King of the Sky, Bane of the Land

[703 rows x 2 columns]
    quest_id                     quest_name
0          1           Elder Fugen's Errand
1          2             Thick, Gooey Dango
2          3         Bunny Dango—with Eggs!
3          4                  A Dango a Day
4          5             Only the Good Eggs
..       ...                            ...
80       146        Resplendent Red Request
81       148                  Bag a 

## Quest Levels & Targets

COLUMNS:
- target_type:
    - Kill: slay target
    - AllMainEnemy: hunt all listed under secondary
    - Capture: capture target
    - ItemGet: collect item
    - Hunting: slay or capture
    - EmTotal: slay small monsters

In [29]:
with open('breakdown/quests.json') as file:
    raw = json.load(file)

# Quest Levels
sections = ['normal_quest_data',
            'normal_quest_data_mr',
            'dl_quest_data',
            'dl_quest_data_mr']
df = pd.DataFrame()
for x in sections:
    sect = raw.get(x).get('param')
    df = pd.concat([df,pd.json_normalize(sect)], axis=0, ignore_index=True)
df = df[['quest_no',
         'quest_level',
         'enemy_level',
         'order_type',
         'quest_type',
         'target_type',
         'tgt_em_type',
         'boss_em_type',
         'auto_match_hr']].reset_index(drop=True)
#large monsters only
df['tgt_em_type'] = df['tgt_em_type'].apply(lambda x: [i['Em'] for i in x if 'Em' in i])
df['tgt_em_type'] = df['tgt_em_type'].apply(lambda x: [i for i in x if i>0])
df['boss_em_type'] = df['boss_em_type'].apply(lambda x: [i['Em'] for i in x])
df['boss_em_type'] = df['boss_em_type'].apply(lambda x: [i for i in x if i>0])

mapper = dict(zip(df_monsters['em_type.Em'], df_monsters['mon_name']))
df['tgt_em_type'] = df['tgt_em_type'].apply(lambda x: [mapper.get(i,0) for i in x])
df['boss_em_type'] = df['boss_em_type'].apply(lambda x: [mapper.get(i,0) for i in x])

df = pd.merge(left=df, left_on='quest_no',
              right=df_quest_names, right_on='quest_id',
              how='left')

'''
Consolidate primary and 2ndary target columns
'''

df['tgt_em_type'] = df.apply(lambda x: x['boss_em_type'] if 'AllMainEnemy' in x['target_type'] else x['tgt_em_type'], axis=1)

# df.rename(columns={'tgt_em_type':'tgt_em_type.Em',
#                    'boss_em_type':'snd_em_type.Em',
#                    'quest_no':'quest_id'}, inplace=True)


# print(df_quests.iloc[604:606])
# df_quests.to_excel('quest.xlsx')
# df.to_excel('raw.xlsx')

In [43]:
'''fill this one out for above cell'''

# Rampage details
sect = raw.get('fixed_hyakuryu_quest')
sections = pd.json_normalize(sect).columns.tolist()
df = pd.DataFrame()
for x in sections:
    df = pd.concat([df,pd.json_normalize(sect.get(x))], axis=0, ignore_index=True)
print(df)

   quest_no  random_seed                               attr  \
0       304            0                   [FIX_WAVE_ORDER]   
1     10216            0                                 []   
2     10403            0                                 []   
3     10603            0                                 []   
4     60109            0                                 []   
5     60113            0                                 []   
6     60116            0  [FIX_WAVE_ORDER, FINAL_BOSS_KILL]   
7     60153            0                                 []   
8     60139            0                                 []   
9     60144            0                   [FIX_WAVE_ORDER]   

                                           wave_data quest_lv  map_no  \
0  [{'boss_em': {'Em': 60}, 'boss_sub_type': 0, '...      QL3       7   
1  [{'boss_em': {'Em': 92}, 'boss_sub_type': 0, '...      QL2       7   
2  [{'boss_em': {'Em': 98}, 'boss_sub_type': 0, '...      QL4       7   
3  [{'boss_em'

In [ ]:
# Quest Unlock by Talk Flag & Misc
sect = raw.get('quest_unlock').get('quest_unlock_by_talk_flag')
df_quest_talk_flag = pd.json_normalize(sect)
df_quest_talk_flag.drop(columns='is_clear', inplace=True)
df_quest_talk_flag.rename(columns={'quest_no':'quest_id'}, inplace=True)

print(df_quest_talk_flag.head())

In [18]:
# Quest Unlock by Quest Clear
sect = raw.get('quest_unlock').get('quest_unlock_by_quest_clear')
df_quest_clear_flag = pd.json_normalize(sect)

In [19]:
# Quest Unlock by Monster Hunt
sect = raw.get('quest_unlock').get('quest_unlock_by_hunt_enemy')
df_quest_hunt_flag = pd.json_normalize(sect)

## progress flags
OUTPUT: **df_prog_flags**
| progress_flag | quest_name | rank |
|-|-|-|

In [ ]:
# Progress Flags
sect = raw.get('progress').get('param_list')
df = pd.json_normalize(sect)
df.fillna(-1,inplace=True)
# df.rename(columns={'progress_flag':'',
#                    'quest_no':'',
#                    'enable_progress_hr_check':'',
#                    'progress_hr':'',
#                    'village.VillageProgress':'',
#                    'hall.HallProgress':'',
#                    'mr.MasterRankProgress':''}, inplace=True)
df = pd.merge(left=df, left_on='quest_no', right=df_quest_names[['quest_id','quest_name']], right_on='quest_id', how='left')
df['village.VillageProgress'] = pd.to_numeric(df['village.VillageProgress'],errors='coerce', downcast='integer')
df['hall.HallProgress'] = pd.to_numeric(df['hall.HallProgress'],errors='coerce', downcast='integer')
df['mr.MasterRankProgress'] = pd.to_numeric(df['mr.MasterRankProgress'],errors='coerce', downcast='integer')
df['rank'] = df.apply(lambda x: [x['village.VillageProgress'],x['hall.HallProgress'],x['mr.MasterRankProgress']], axis=1)
# df_prog_flags = df[['progress_flag','quest_no','quest_name','enable_progress_hr_check','progress_hr','rank']].copy()
df_prog_flags = df[['progress_flag','talk_flag','quest_name','progress_hr','rank']].copy()

#talk flags
df_prog_flags = pd.merge(left=df_prog_flags, left_on='talk_flag', right=df_quest_talk_flag, right_on='talk_flag', how='left')
df_prog_flags.drop(columns='talk_flag', inplace=True)
# df_prog_flags.rename(columns={'quest_name':'quest_name_1'}, inplace=True)
df_prog_flags = pd.merge(left=df_prog_flags, left_on='quest_id', right=df_quest_names[['quest_id','quest_name']], right_on='quest_id', how='left')
df_prog_flags.drop(columns='quest_id', inplace=True)

df_prog_flags ['quest_name'] = df_prog_flags['quest_name_x'].fillna(df_prog_flags['quest_name_y'])
df_prog_flags = df_prog_flags[['progress_flag','quest_name','rank']]

print(df_prog_flags.head())
# df_prog_flags.to_excel('prog_flags.xlsx')

# Skills

## Skill Name & ID

SECTIONS:
- **player_skill_name_msg**
- **player_skill_name_msg_mr**

COLUMNS:
- **skill_id**: armor skill internal id. HR: 0-199, MR: 200-235, Rampage MR: 259-293
- **skill_name**: armor skill name

OUTPUT: **df_skill_names**
| skill_id | skill_name |
|-|-|
| 0 | Attack Boost |
| 1 | Agitator |
| 2 | Peak Performance |
| 3 | Resentment |
| 4 | Resuscitate |

In [ ]:
with open('breakdown/skills.json') as file:
    raw = json.load(file)

# Skill Names & IDs
sections = ['player_skill_name_msg',
            'player_skill_name_msg_mr',
            # 'hyakuryu_skill_name_msg',
            'hyakuryu_skill_name_msg_mr']
df_skill_names = pd.DataFrame()
for x in sections:
    sect = raw.get(x).get('entries')
    df = pd.json_normalize(sect)
    df['skill_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
    df['skill_id'] = df['name'].str.extract('(\d+)', expand=False) # grab id numbers
    df = df[df['skill_name']!=''] # remove blank names
    df.dropna(inplace=True) # remove blank ids
    df = df[~df['skill_name'].str.contains('COLOR FF0000', na=False)]
    df['skill_id'] = pd.to_numeric(df['skill_id'], errors='ignore', downcast='integer')
    df.reset_index(drop=True, inplace=True)
    df_skill_names = pd.concat([df_skill_names,df[['skill_id', 'skill_name']]], axis=0, ignore_index=True)
df_skill_names.reset_index(drop=True, inplace=True)

print(df_skill_names.head())

# Armor Names & IDs

OUTPUT: df_armors
| series_id | armor_type | armor_name | series_name |
|-|-|-|-|
| 0 | Head | Kamura Head Scarf | Kamura |
| 1 | Head | Kamura Head Scarf S | Kamura S |
| 2 | Head | Leather Headgear | Leather |
| 3 | Head | Leather Headgear S | Leather S |
| 4 | Head | Chainmail Headgear | Chainmail |

In [ ]:
with open('breakdown/armors.json') as file:
    raw = json.load(file)

# Armor Names & IDs
sections = [
    'armor_head_name_msg',
    'armor_chest_name_msg',
    'armor_arm_name_msg',
    'armor_waist_name_msg',
    'armor_leg_name_msg',
    'armor_head_name_msg_mr',
    'armor_chest_name_msg_mr',
    'armor_arm_name_msg_mr',
    'armor_waist_name_msg_mr',
    'armor_leg_name_msg_mr'
]
df_armor_names = pd.DataFrame()
for x in sections:
    sect = raw.get(x).get('entries')
    df = pd.json_normalize(sect)
    df_armor_names = pd.concat([df_armor_names,df],axis=0,ignore_index=True)


df_armor_names['armor_name'] = df_armor_names['content'].apply(lambda x: x[1] if not x[1]=='' else np.nan)
df_armor_names = df_armor_names[~df_armor_names['armor_name'].str.contains('COLOR FF0000', na=False)]
df_armor_names.dropna(subset=['armor_name'], inplace=True)
df_armor_names['series_id'] = df_armor_names['name'].str.extract('(\d+)', expand=False)
df_armor_names['series_id'] = pd.to_numeric(df_armor_names['series_id'], errors='coerce', downcast='integer')
df_armor_names['armor_type'] = df_armor_names['name'].str.extract('A_([^_]+)', expand=False)

df_armor_names = df_armor_names[['series_id','armor_type','armor_name']]
df_armor_names.reset_index(drop=True, inplace=True)

sections = ['armor_series_name_msg','armor_series_name_msg_mr']
df_armor_series = pd.DataFrame()
for x in sections:
    sect = raw.get(x).get('entries')
    df = pd.json_normalize(sect)
    df_armor_series = pd.concat([df_armor_series,df],axis=0,ignore_index=True)

df_armor_series['series_id'] = df_armor_series['name'].str.extract('(\d+)', expand=False)
df_armor_series['series_name'] = df_armor_series['content'].apply(lambda x: x[1] if not x[1]=='' else np.nan)
df_armor_series = df_armor_series[~df_armor_series['series_name'].str.contains('COLOR FF0000', na=False)]
df_armor_series.dropna(subset=['series_name'], inplace=True)
df_armor_series['series_id'] = pd.to_numeric(df_armor_series['series_id'], errors='coerce', downcast='integer')

df_armor_series = df_armor_series[['series_id','series_name']]
df_armor_series.reset_index(drop=True,inplace=True)

df_armors = pd.merge(left=df_armor_names, left_on='series_id',
                     right=df_armor_series,right_on='series_id',how='left')

print(df_armors.head())
# df_armor_names.to_excel('armor_names.xlsx')

## Gender Specific Armor Name Pairs

SECTIONS:
- **armor_pair**

COLUMNS:
- **male_armor_id**
- **female_armor_id**
- **male_armor_name**
- **female_armor_name**

OUTPUT: **df_armor_pair**

In [ ]:
# Armor Pair
sect = raw.get('armor_pair').get('param')
df = pd.json_normalize(sect)

print(df)

## Armor Unlock Conditions

OUTPUT: **df_armor_flags**

In [ ]:
# Armor Unlock Conditions
sect = raw.get('armor_product').get('param')
df = pd.json_normalize(sect)

sections = ['id.Head', 'id.Chest', 'id.Arm', 'id.Waist', 'id.Leg']
df['series_id']=np.nan
for y in sections:
    df['series_id'].fillna(df[y], inplace=True)
df['series_id'] = pd.to_numeric(df['series_id'], errors='coerce',downcast='integer')
df['armor_type']=np.nan
for y in sections:
    df['armor_type'] = df.apply(lambda x: y[3:] if pd.notna(x[y]) else x['armor_type'], axis=1)
df['enemy_flag.Em'].fillna(df['enemy_flag.Ems'], inplace=True)
df['item'] = df['item'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['item_num'] = df['item_num'].apply(lambda y: [x for x in y if x>0])

df = pd.merge(left=df, left_on=['series_id', 'armor_type'],
              right=df_armors, right_on=['series_id', 'armor_type'],
              how='left')

df.fillna(-1, inplace=True)
df['enemy_flag.Em'] = pd.to_numeric(df['enemy_flag.Em'], errors='coerce', downcast='integer')
df['item_flag.Normal'] = pd.to_numeric(df['item_flag.Normal'], errors='coerce', downcast='integer')

df.rename(columns={'item':'craft_item_id_list',
                   'item_num':'craft_num_list',
                   'enemy_flag.Em':'key_em_type.Em',
                   'item_flag.Normal':'key_item_id'}, inplace=True)
df_armor_flags = df[['series_name','armor_type','armor_name','craft_item_id_list','craft_num_list','progress_flag','key_em_type.Em','key_item_id']].copy()
# df_armor_flags = df
df_armor_flags.reset_index(drop=True, inplace=True)

print(df_armor_flags.head())

In [ ]:
# human readable armor flags
df_armor_flags = pd.merge(left=df_armor_flags, left_on='progress_flag',
                          right=df_prog_flags, right_on='progress_flag',
                          how='left')
df_armor_flags.drop(columns='progress_flag',axis=1, inplace=True)

df_armor_flags = pd.merge(left=df_armor_flags, left_on='key_em_type.Em',
                          right=df_monsters[['em_type.Em','mon_name']], right_on='em_type.Em',
                          how='left')
df_armor_flags.drop(columns=['key_em_type.Em','em_type.Em'], axis=1, inplace=True)

df_armor_flags = pd.merge(left=df_armor_flags, left_on='key_item_id',
                          right=df_items[['item_id','item_name']], right_on='item_id',
                          how='left')
df_armor_flags.drop(columns=['key_item_id', 'item_id'], axis=1, inplace=True)

print(df_armor_flags.head())
# df_armor_flags.to_excel('armor_flags.xlsx')

# Decorations

## Schema

### "decorations": param
- **decorations**: "id":{"Deco": 11}
- **decoration_lv**: 2
- **skill_id_list**: [{"Skill": 11},"None"]
- **skill_lv_list**: [1,0]

### "decorations_product": param
- **id**: {"Deco": 0}
- **item_flag**: "None"
- **enemy_flag**: {"Em": 99}
- **progress_flag**: 0
- **item_id_list**: [{"Normal": 1036},
                     {"Normal": 337},
                     {"Normal": 571},
                     {"Normal": 179}]
- **item_num_list**: [5,3,3,1]

### "decorations_name_msg":entries
- **name**: "Decorations_000_Name
- **content**: ["...", "Attack Jewel 2", "..."]

### "decorations_name_msg_mr":entries
- **name**: "Decorations_201_Name
- **content**: ["...", "Hard Fire Res Jewel 4", "..."]

## Deco Names, Skills, & IDs
REFERENCES:
- **df_skill_names**

SECTIONS:
- **decoration_name_msg**
- **decorations_name_msg_mr**
- **decorations**

COLUMNS:
- **deco_id**: internal deco id number. HR: 0-199, MR: 200-352, Rampage: 400-434
- **deco_name**: decoration name
- **deco_size**: decoration size
- **skill_name**: name of skill that decoration provides

OUTPUT: **df_decos**
| deco_id | deco_name | deco_size | skill_name |
|-|-|-|-|
| 0 | Attack Jewel 2 | 2 | Attack Boost |
| 1 | Challenger Jewel 2 | 2 | Agitator |
| 2 | Flawless Jewel 2 | 2 | Peak Performance |
| 3 | Furor Jewel 2 | 2 | Resentment |
| 4 | Crisis Jewel 2 | 2 | Resuscitate |

In [ ]:
with open('breakdown/decos.json') as file:
    raw = json.load(file)

# Decoration Names
deco_name_list = ['decorations_name_msg','decorations_name_msg_mr','hyakuryu_decos_name_msg']
df_deco_names = pd.DataFrame()
for x in deco_name_list:
    sect = raw.get(x).get('entries')
    df = pd.json_normalize(sect)
    df['deco_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
    df['deco_id'] = df['name'].str.extract('(\d+)', expand=False) # grab id numbers
    df.dropna(inplace=True) # remove blank ids
    df['deco_id'] = pd.to_numeric(df['deco_id'], errors='ignore', downcast='integer')
    if x == 'hyakuryu_decos_name_msg':
        df['deco_id'] = df['deco_id']+400
    df = df[df['deco_name']!=''] # remove blank names
    df = df[~df['deco_name'].str.contains('COLOR FF0000', na=False)]
    df_deco_names = pd.concat([df_deco_names, df[['deco_id', 'deco_name']]], axis=0, ignore_index=True)
df_deco_names.reset_index(drop=True, inplace=True)

# print(df_deco_names)

# Decoration Skills
sect = raw.get('decorations').get('param')
df = pd.json_normalize(sect)
df['skill_grade'] = df['skill_id_list'].apply(lambda x: next(iter(x[0]))) # grab MR category
df['skill_id'] = df['skill_id_list'].apply(lambda x: next(iter(x[0].values()))) # grab skill ID
df.loc[df['skill_grade']=='MrSkill', 'skill_id'] += 200 # convert skill ID to +200 for MR
df['id.Deco'].fillna(df['id.MrDeco']+200, inplace=True) # merge MrDeco ID into Deco IDs
df.rename(columns={'id.Deco':'deco_id',
                'decoration_lv':'deco_size'}, inplace=True)
df_deco_skills = df[['deco_id','deco_size','skill_id']].copy()

#Rampage Decoration Skills
sect = raw.get('hyakuryu_decos').get('param')
df = pd.json_normalize(sect)
df['id.Deco'] = df['id.Deco']+400
df.rename(columns={'id.Deco':'deco_id',
                'decoration_lv':'deco_size',
                'hyakuryu_skill_id.Skill':'skill_id'}, inplace=True)
df_deco_skills = pd.concat([df_deco_skills, df[['deco_id','deco_size','skill_id']]], axis=0, ignore_index=True)
df_deco_skills['deco_id'] = pd.to_numeric(df_deco_skills['deco_id'], errors='coerce', downcast='integer')
df_deco_skills.reset_index(drop=True, inplace=True)

# print(df_deco_skills)

#
df_decos = pd.DataFrame()
# df_deco_skills = pd.merge(left=df[['deco_id','deco_size','skill_id']], left_on='skill_id', right=df_skill_names, right_on='skill_id', how='left')
df_decos = pd.merge(left=df_deco_skills, left_on='skill_id', right=df_skill_names, right_on='skill_id', how='left')
df_decos.drop(columns=['skill_id'], inplace=True)

df_decos = pd.merge(left=df_deco_names, left_on='deco_id', right=df_decos, right_on='deco_id', how='left')
df_decos.dropna(axis=0, inplace=True)
df_decos['deco_size'] = pd.to_numeric(df_decos['deco_size'], errors='coerce',downcast='integer')
# print(df_deco_skills)

print(df_decos.head())

## Deco Unlock Conditions

SECTIONS:
- **decorations_product**

COLUMNS:
- **

OUTPUT:
| deco_id | craft_item_id_list | craft_num_list | progress_flag | key_em_type.Em | key_item_id |
|-|-|-|-|-|-|
| 0 | [1036, 337, 571, 179] | [5, 3, 3, 1] | 0 | 99 | -1 |
| 1 | [1037, 645, 560, 904] | [4, 2, 3, 1] | 3 | 0 | -1 |   
| 2 | [1037, 604, 581, 281] | [4, 2, 3, 1] | 3 | 1874 | -1 |  
| 3 | [1037, 330, 266, 578] | [4, 2, 3, 2] | 3 | 0 | -1 |
| 4 | [1037, 285, 327, 904] | [4, 3, 2, 1] | 3 | 0 | -1 |

In [29]:
# Decoration Unlock Conditions

sect = raw.get('decorations_product').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['item_num_list'] = df['item_num_list'].apply(lambda y: [x for x in y if x>0])
df['id.Deco'].fillna(df['id.MrDeco']+200, inplace=True) # merge MrDeco ID into Deco IDs
df['item_flag.Normal'].fillna(-1, inplace=True)
df.rename(columns={'id.Deco':'deco_id',
                   'enemy_flag.Em':'key_em_type.Em',
                   'item_flag.Normal':'key_item_id',
                   'item_id_list':'craft_item_id_list',
                   'item_num_list':'craft_num_list'}, inplace=True)
df_deco_flags = df[['deco_id','craft_item_id_list','craft_num_list','progress_flag','key_em_type.Em','key_item_id']].copy()
df_deco_flags['deco_id'] = pd.to_numeric(df_deco_flags['deco_id'], errors='ignore', downcast='integer')
df_deco_flags['key_item_id'] = pd.to_numeric(df_deco_flags['key_item_id'], errors='ignore', downcast='integer')

# Rampage Decoration Unlock Conditions
sect = raw.get('hyakuryu_decos_product').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['item_num_list'] = df['item_num_list'].apply(lambda y: [x for x in y if x>0])
df['id.Deco'] = df['id.Deco']+400
df['item_flag'].fillna(-1, inplace=True)
df.rename(columns={'id.Deco':'deco_id',
                   'enemy_flag.Em':'key_em_type.Em',
                   'item_flag':'key_item_id',
                   'item_id_list':'craft_item_id_list',
                   'item_num_list':'craft_num_list'}, inplace=True)
df_deco_flags = pd.concat([df_deco_flags,df[['deco_id','craft_item_id_list','craft_num_list','progress_flag','key_em_type.Em','key_item_id']]], axis=0, ignore_index=True)
df_deco_flags['deco_id'] = pd.to_numeric(df_deco_flags['deco_id'], errors='ignore', downcast='integer')
df_deco_flags['key_item_id'] = pd.to_numeric(df_deco_flags['key_item_id'], errors='ignore', downcast='integer')
df_deco_flags.reset_index(drop=True, inplace=True)
# print(df_deco_flags)

In [30]:
# decoding deco flags
df = df_deco_flags
df = pd.merge(left=df, left_on='progress_flag', right=df_prog_flags, right_on='progress_flag', how='left')
df.drop(columns=['progress_flag'], inplace=True)
df = pd.merge(left=df, left_on='key_item_id', right=df_items[['item_id','item_name']], right_on='item_id', how='left')
df.drop(columns=['key_item_id','item_id'], inplace=True)
df = pd.merge(left=df, left_on='key_em_type.Em', right=df_monsters[['em_type.Em','mon_name']], right_on='em_type.Em', how='left')
df.drop(columns=['key_em_type.Em','em_type.Em'], inplace=True)
df.fillna(-1, inplace=True)

df = pd.merge(left=df, left_on='deco_id', right=df_deco_skills, right_on='deco_id', how='left')
df = pd.merge(left=df, left_on='deco_id', right=df_deco_names, right_on='deco_id', how='left')
df = pd.merge(left=df, left_on='skill_id', right=df_skill_names, right_on='skill_id', how='left')

df_deco_unlock = df.copy()

# df_deco_unlock.to_excel('deco_unlock.xlsx')
# print(df_deco_unlock)

In [ ]:
# '''Gathering Node Drop Tables'''
# sect = raw.get('item_pop_lot').get('param')
# df = pd.json_normalize(sect)
# df['lower_item_ids'] = df['lower_id'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
# df['lower_item_ct'] = df['lower_num'].apply(lambda y: [x for x in y if x>0])
# df['lower_item_chance'] = df['lower_probability'].apply(lambda y: [x for x in y if x>0])
# df['upper_item_ids'] = df['upper_id'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
# df['upper_item_ct'] = df['upper_num'].apply(lambda y: [x for x in y if x>0])
# df['upper_item_chance'] = df['upper_probability'].apply(lambda y: [x for x in y if x>0])
# df['master_item_ids'] = df['master_id'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
# df['master_item_ct'] = df['master_num'].apply(lambda y: [x for x in y if x>0])
# df['master_item_chance'] = df['master_probability'].apply(lambda y: [x for x in y if x>0])
# df_item_pop_lot = df[['pop_id','field_type','lot_count','lower_item_ids','lower_item_ct', 'lower_item_chance','upper_item_ids','upper_item_ct', 'upper_item_chance','master_item_ids','master_item_ct', 'master_item_chance']]
# print(df_item_pop_lot.head(5))
# df_item_pop_lot.to_excel('gathering_node_drop_lot.xlsx')